<a href="https://colab.research.google.com/github/sda96/AIFFEL_3rd_hackerton_TUNiB_DKTC/blob/main/notebook/SeungTaek/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 13s (758 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155229 files and di

In [2]:
!gdown https://drive.google.com/uc?id=1A8IdrqgEjRhvg9KoV6dopo5GirTKO3aj
!gdown https://drive.google.com/uc?id=1SCxtUqlGEzYCfCbHUREFf5UqoM1uWDBK
!gdown https://drive.google.com/uc?id=1Zh4mCpgQReE4E0sAjiFXwSV6Y7lc77fO
!gdown https://drive.google.com/uc?id=19jT2ZTb6oj6sFERkl5AT3qQExxN3D5cK

Downloading...
From: https://drive.google.com/uc?id=1A8IdrqgEjRhvg9KoV6dopo5GirTKO3aj
To: /content/sns_val.zip
100% 34.8M/34.8M [00:01<00:00, 31.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SCxtUqlGEzYCfCbHUREFf5UqoM1uWDBK
To: /content/sns_train.zip
100% 278M/278M [00:04<00:00, 65.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Zh4mCpgQReE4E0sAjiFXwSV6Y7lc77fO
To: /content/train.csv
100% 2.24M/2.24M [00:00<00:00, 185MB/s]
Downloading...
From: https://drive.google.com/uc?id=19jT2ZTb6oj6sFERkl5AT3qQExxN3D5cK
To: /content/test.json
100% 417k/417k [00:00<00:00, 114MB/s]


In [3]:
!unzip -uq "/content/sns_train.zip" -d "/content/sns_data"
!unzip -uq "/content/sns_val.zip" -d "/content/sns_data"

In [4]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 14.2 MB/s 
     |████████████████████████████████| 895 kB 64.3 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 6.8 MB 53.1 MB/s 
     |████████████████████████████████| 596 kB 61.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import pandas as pd # pandas 모듈 로드
import json  # json 모듈 로드
import re
import os
from glob import glob
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from collections import defaultdict, Counter

import tensorflow as tf
from transformers import TextClassificationPipeline
from transformers import BertTokenizer, TFBertForSequenceClassification   #BERT
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification   #RoBerta

In [6]:
train_df = pd.read_csv("/content/train.csv")

In [7]:
train_df

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
3945,3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,3948,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [8]:
data_path ="/content/sns_data/content/sns_csv_train"
files = glob(data_path + "/*")
files

['/content/sns_data/content/sns_csv_train/상거래(쇼핑).csv',
 '/content/sns_data/content/sns_csv_train/개인및관계.csv',
 '/content/sns_data/content/sns_csv_train/행사.csv',
 '/content/sns_data/content/sns_csv_train/시사교육.csv',
 '/content/sns_data/content/sns_csv_train/미용과건강.csv',
 '/content/sns_data/content/sns_csv_train/일과직업.csv',
 '/content/sns_data/content/sns_csv_train/주거와생활.csv',
 '/content/sns_data/content/sns_csv_train/식음료.csv',
 '/content/sns_data/content/sns_csv_train/여가생활.csv']

In [9]:
df_all = pd.DataFrame()
for i in files:
    df = pd.read_csv(i)
    df_all = pd.concat([df_all, df])

In [10]:
df_all

,0
0,맥주삿당 역시 태풍엔 맥주지\n오 뭐뭐 샀음?\n필라이트 좡색\n오 나는 구스 아일...
1,일단 니 에스쁘아쿠션까지해서 총6개\n에스쁘아쿠션사게?̊̈ ㅋㅋㅋㅋㅋ\n아니 니 어...
2,누리카드사용 낼부터된다네 책을 좀 빨리봐야해서 오늘 그냥주문해줘\n알겠음 집으로 시...
3,언니 텐텐서 컵 살거야? 살 거면 나 테이블보? 무슨 패브릭 이런거 하나만 사게 업...
4,하 어쩌냐 팔기싫다 또 질문해 어찌고저찌고\nㅋㅋㅋㅋㅋㅋㅋ 아 또 뭐래...\n블투...
...,...
190310,그래서 나는 국대경기를 젤 좋아해\n나 무슨 마음인지\n내가 제일 좋아하는 선수들만...
190311,. .\n좋은겅가ㅎㅎㅎㅎㅎㅎㅎㅎ\n그래. . 나한테만하지말고 다른데도 좀 써먹어. ...
190312,근데 요즘 비둘기들 진짜 겁대가리상실\nㅋㅋㅋㅋㅋㅋㅋ겁대가리 상실ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
190313,#@계정# 토스 퀴즈 정답 0825\n오올~~~ 141원 들어왔어요^^\n오오 ㅋ...


In [11]:
df_all["class"] = "일반 대화"

In [12]:
df_all.head()

,0,class
0,맥주삿당 역시 태풍엔 맥주지\n오 뭐뭐 샀음?\n필라이트 좡색\n오 나는 구스 아일...,일반 대화
1,일단 니 에스쁘아쿠션까지해서 총6개\n에스쁘아쿠션사게?̊̈ ㅋㅋㅋㅋㅋ\n아니 니 어...,일반 대화
2,누리카드사용 낼부터된다네 책을 좀 빨리봐야해서 오늘 그냥주문해줘\n알겠음 집으로 시...,일반 대화
3,언니 텐텐서 컵 살거야? 살 거면 나 테이블보? 무슨 패브릭 이런거 하나만 사게 업...,일반 대화
4,하 어쩌냐 팔기싫다 또 질문해 어찌고저찌고\nㅋㅋㅋㅋㅋㅋㅋ 아 또 뭐래...\n블투...,일반 대화


In [13]:
df_all = df_all.rename(columns = {'0' : "conversation"})

In [14]:
df_all

,conversation,class
0,맥주삿당 역시 태풍엔 맥주지\n오 뭐뭐 샀음?\n필라이트 좡색\n오 나는 구스 아일...,일반 대화
1,일단 니 에스쁘아쿠션까지해서 총6개\n에스쁘아쿠션사게?̊̈ ㅋㅋㅋㅋㅋ\n아니 니 어...,일반 대화
2,누리카드사용 낼부터된다네 책을 좀 빨리봐야해서 오늘 그냥주문해줘\n알겠음 집으로 시...,일반 대화
3,언니 텐텐서 컵 살거야? 살 거면 나 테이블보? 무슨 패브릭 이런거 하나만 사게 업...,일반 대화
4,하 어쩌냐 팔기싫다 또 질문해 어찌고저찌고\nㅋㅋㅋㅋㅋㅋㅋ 아 또 뭐래...\n블투...,일반 대화
...,...,...
190310,그래서 나는 국대경기를 젤 좋아해\n나 무슨 마음인지\n내가 제일 좋아하는 선수들만...,일반 대화
190311,. .\n좋은겅가ㅎㅎㅎㅎㅎㅎㅎㅎ\n그래. . 나한테만하지말고 다른데도 좀 써먹어. ...,일반 대화
190312,근데 요즘 비둘기들 진짜 겁대가리상실\nㅋㅋㅋㅋㅋㅋㅋ겁대가리 상실ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,일반 대화
190313,#@계정# 토스 퀴즈 정답 0825\n오올~~~ 141원 들어왔어요^^\n오오 ㅋ...,일반 대화


In [15]:
df_all = df_all[['class', 'conversation']]
df_all

,class,conversation
0,일반 대화,맥주삿당 역시 태풍엔 맥주지\n오 뭐뭐 샀음?\n필라이트 좡색\n오 나는 구스 아일...
1,일반 대화,일단 니 에스쁘아쿠션까지해서 총6개\n에스쁘아쿠션사게?̊̈ ㅋㅋㅋㅋㅋ\n아니 니 어...
2,일반 대화,누리카드사용 낼부터된다네 책을 좀 빨리봐야해서 오늘 그냥주문해줘\n알겠음 집으로 시...
3,일반 대화,언니 텐텐서 컵 살거야? 살 거면 나 테이블보? 무슨 패브릭 이런거 하나만 사게 업...
4,일반 대화,하 어쩌냐 팔기싫다 또 질문해 어찌고저찌고\nㅋㅋㅋㅋㅋㅋㅋ 아 또 뭐래...\n블투...
...,...,...
190310,일반 대화,그래서 나는 국대경기를 젤 좋아해\n나 무슨 마음인지\n내가 제일 좋아하는 선수들만...
190311,일반 대화,. .\n좋은겅가ㅎㅎㅎㅎㅎㅎㅎㅎ\n그래. . 나한테만하지말고 다른데도 좀 써먹어. ...
190312,일반 대화,근데 요즘 비둘기들 진짜 겁대가리상실\nㅋㅋㅋㅋㅋㅋㅋ겁대가리 상실ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
190313,일반 대화,#@계정# 토스 퀴즈 정답 0825\n오올~~~ 141원 들어왔어요^^\n오오 ㅋ...


In [16]:
df_train2 = df_all.sample(n = 900, random_state = 5)

In [17]:
df_train = pd.concat([train_df, df_train2])
df_train

,idx,class,conversation
0,0.0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1.0,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2.0,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3.0,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4.0,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
482482,NaN,일반 대화,돌아왔슴당\n응응 ㅋㅋㅋㅋㅋㅋㅋㅋ 내가? 앖어지고 ㅋㅋㅋ\n내가 없어지고 여치니사진...
4376,NaN,일반 대화,마쟈!!!!!! 호엥잉~\n신났누!!!\n근데 비싼거살때 깔끔한거 사야\n복지몰가능...
138889,NaN,일반 대화,오잉\n#@시스템#사진#\n#@이름# 잘때 다 그릿네 잘그릿단\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
35622,NaN,일반 대화,피니가 시른게 아니구 피니는 맨날\n핀네일에 심플해죠 해\n모를 올려줄라 하자나 피...


In [18]:
df_train = df_train.drop(['idx'],axis=1)
df_train

,class,conversation
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
482482,일반 대화,돌아왔슴당\n응응 ㅋㅋㅋㅋㅋㅋㅋㅋ 내가? 앖어지고 ㅋㅋㅋ\n내가 없어지고 여치니사진...
4376,일반 대화,마쟈!!!!!! 호엥잉~\n신났누!!!\n근데 비싼거살때 깔끔한거 사야\n복지몰가능...
138889,일반 대화,오잉\n#@시스템#사진#\n#@이름# 잘때 다 그릿네 잘그릿단\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
35622,일반 대화,피니가 시른게 아니구 피니는 맨날\n핀네일에 심플해죠 해\n모를 올려줄라 하자나 피...


In [19]:
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,class,conversation
0,협박 대화,이봐. 아저씨. 차를 이렇게 주차하면 어떡해?\n아니 언제 봤다고 반말이야?\n기분...
1,일반 대화,김치볶음밥 먹었어 헤헤\n케케ㅋㅋㅋㅋ자랫서어 반찬마싯게해서 더먹지왜엥\n계란 올려먹...
2,직장 내 괴롭힘 대화,야이 신입이 너랑 나랑 일하는게 이렇게 천지차인데 월급은 겨우 이정도 차이냐?\n죄...
3,일반 대화,기두르\nㅋㅋㅋㅋ긔엽네\n#@시스템#사진# 엄마착샷 ㅋㅋㅋ\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
4,갈취 대화,저기 윤대리 그 연필 꽂이에 있는 펜 내 거 아니야?\n아닌데요. 이거 제가 산 펜...
...,...,...
4845,기타 괴롭힘 대화,여기 애기 먹을 공기밥좀 주세요\n아네 공기밥 하나 추가해드릴까요?\n아니 추가 말...
4846,기타 괴롭힘 대화,야 넌 평생 못걷냐?\n응 어렸을 때 사고가 나서\n헐 불쌍해 그럼 이거 의족이냐?...
4847,협박 대화,뭘 꼬라봐?\n 왜 시비세요?\n 끝까지 꼬라보네? 눈깔 뽑히고 싶냐?\n 아니 한...
4848,직장 내 괴롭힘 대화,사장님 이번에 새로 들어온 친구에요\n안녕하십니까\n어 일잘하게 생겼네.군대는 갔다...


In [20]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣0-9?.!,\n]+", " ", sentence)

    sentence = sentence.strip()

    return sentence

In [21]:
df_train["conversation"] = df_train["conversation"].apply(preprocess_sentence)
df_train

,class,conversation
0,협박 대화,이봐 . 아저씨 . 차를 이렇게 주차하면 어떡해 ? \n아니 언제 봤다고 반말이야 ...
1,일반 대화,김치볶음밥 먹었어 헤헤\n케케ㅋㅋㅋㅋ자랫서어 반찬마싯게해서 더먹지왜엥\n계란 올려먹...
2,직장 내 괴롭힘 대화,야이 신입이 너랑 나랑 일하는게 이렇게 천지차인데 월급은 겨우 이정도 차이냐 ? \...
3,일반 대화,기두르\nㅋㅋㅋㅋ긔엽네\n 시스템 사진 엄마착샷 ㅋㅋㅋ\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
4,갈취 대화,저기 윤대리 그 연필 꽂이에 있는 펜 내 거 아니야 ? \n아닌데요 . 이거 제가 ...
...,...,...
4845,기타 괴롭힘 대화,여기 애기 먹을 공기밥좀 주세요\n아네 공기밥 하나 추가해드릴까요 ? \n아니 추가...
4846,기타 괴롭힘 대화,야 넌 평생 못걷냐 ? \n응 어렸을 때 사고가 나서\n헐 불쌍해 그럼 이거 의족이...
4847,협박 대화,뭘 꼬라봐 ? \n 왜 시비세요 ? \n 끝까지 꼬라보네 ? 눈깔 뽑히고 싶냐 ? ...
4848,직장 내 괴롭힘 대화,사장님 이번에 새로 들어온 친구에요\n안녕하십니까\n어 일잘하게 생겼네 . 군대는 ...


In [22]:
bucket_class = []
label_names = ["협박 대화", "갈취 대화", "직장 내 괴롭힘 대화", 
               "기타 괴롭힘 대화", "일반 대화"]

for idx, label in enumerate(df_train["class"]):
  if label == label_names[0]:
    bucket_class.append(0)
  elif label == label_names[1]:
    bucket_class.append(1)
  elif label == label_names[2]:
    bucket_class.append(2)
  elif label == label_names[3]:
    bucket_class.append(3)
  elif label == label_names[4]:
    bucket_class.append(4)

df_train["class"] = bucket_class
df_train.head()

,class,conversation
0,0,이봐 . 아저씨 . 차를 이렇게 주차하면 어떡해 ? \n아니 언제 봤다고 반말이야 ...
1,4,김치볶음밥 먹었어 헤헤\n케케ㅋㅋㅋㅋ자랫서어 반찬마싯게해서 더먹지왜엥\n계란 올려먹...
2,2,야이 신입이 너랑 나랑 일하는게 이렇게 천지차인데 월급은 겨우 이정도 차이냐 ? \...
3,4,기두르\nㅋㅋㅋㅋ긔엽네\n 시스템 사진 엄마착샷 ㅋㅋㅋ\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
4,1,저기 윤대리 그 연필 꽂이에 있는 펜 내 거 아니야 ? \n아닌데요 . 이거 제가 ...


In [23]:
X = df_train["conversation"].to_list()
Y = df_train["class"].to_list()

x_train, x_val_test, y_train, y_val_test = train_test_split(X, Y, 
                                                            test_size = 0.1, 
                                                            random_state = 200,
                                                            stratify = Y)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test,
                                                test_size = 0.5,
                                                random_state = 202,
                                                stratify = y_val_test)
print(f"x_train 개수 : {len(x_train)}")
print(f"y_train 개수 : {len(y_train)}")
print(f"x_val 개수 : {len(x_val)}")
print(f"y_val 개수 : {len(y_val)}")
print(f"x_test 개수 : {len(x_test)}")
print(f"y_test 개수 : {len(y_test)}")

print(f"훈련 데이터 레이블 비율 확인 : {Counter(y_train)}")
print(f"검증 데이터 레이블 비율 확인 : {Counter(y_val)}")
print(f"테스트 데이터 레이블 비율 확인 : {Counter(y_test)}")

x_train 개수 : 4365
y_train 개수 : 4365
x_val 개수 : 242
y_val 개수 : 242
x_test 개수 : 243
y_test 개수 : 243
훈련 데이터 레이블 비율 확인 : Counter({3: 985, 1: 883, 2: 881, 4: 810, 0: 806})
검증 데이터 레이블 비율 확인 : Counter({3: 54, 2: 49, 1: 49, 4: 45, 0: 45})
테스트 데이터 레이블 비율 확인 : Counter({3: 55, 1: 49, 2: 49, 0: 45, 4: 45})


In [24]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
kbert_tokenizer = BertTokenizer.from_pretrained("klue/bert-base", use_fast=True)
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base", use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [25]:
bert_train_encodings = bert_tokenizer(x_train, truncation=True, padding=True)
bert_val_encodings = bert_tokenizer(x_val, truncation=True, padding=True)

kbert_train_encodings = kbert_tokenizer(x_train, truncation=True, padding=True)
kbert_val_encodings = kbert_tokenizer(x_val, truncation=True, padding=True)

roberta_train_encodings = roberta_tokenizer(x_train, truncation=True, padding=True)
roberta_val_encodings = roberta_tokenizer(x_val, truncation=True, padding=True)

In [26]:
for key in bert_train_encodings:
  print(f"{key} : {bert_train_encodings[key][1]}")

input_ids : [101, 100, 100, 1459, 30006, 30021, 29992, 30006, 29997, 30008, 100, 1460, 30006, 30024, 29995, 30008, 30020, 29991, 30011, 1455, 30017, 29994, 30019, 29991, 30017, 29994, 30019, 1455, 30014, 29994, 30008, 30023, 100, 100, 1464, 30008, 29992, 30010, 30020, 29995, 30008, 29991, 30008, 29999, 30013, 30025, 100, 1459, 30015, 1461, 30008, 30000, 30008, 30023, 29999, 30017, 30022, 1455, 30006, 29993, 30011, 100, 1463, 30019, 29995, 30011, 30003, 30019, 30002, 30011, 30021, 100, 100, 100, 1463, 30006, 29996, 30008, 29992, 30019, 30023, 100, 100, 1463, 30014, 30023, 30000, 30019, 30020, 29999, 30019, 29995, 30010, 30021, 100, 100, 999, 1029, 1463, 30017, 29999, 30017, 30025, 1459, 30006, 30000, 30006, 1455, 30017, 29994, 30008, 29995, 30010, 30021, 100, 999, 1463, 30017, 29999, 30017, 30025, 29999, 30017, 1455, 30017, 29994, 30008, 29995, 30010, 30021, 100, 1015, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [27]:
bert_train_dataset = tf.data.Dataset.from_tensor_slices((dict(bert_train_encodings), 
                                                    y_train))
bert_val_dataset = tf.data.Dataset.from_tensor_slices((dict(bert_val_encodings), 
                                                  y_val))

kbert_train_dataset = tf.data.Dataset.from_tensor_slices((dict(kbert_train_encodings), 
                                                    y_train))
kbert_val_dataset = tf.data.Dataset.from_tensor_slices((dict(kbert_val_encodings), 
                                                  y_val))

robert_train_dataset = tf.data.Dataset.from_tensor_slices((dict(roberta_train_encodings), 
                                                    y_train))
robert_val_dataset = tf.data.Dataset.from_tensor_slices((dict(roberta_val_encodings), 
                                                  y_val))

In [28]:
kbert_train_dataset

<TensorSliceDataset shapes: ({input_ids: (458,), token_type_ids: (458,), attention_mask: (458,)}, ()), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>

In [29]:
bert_val_dataset

<TensorSliceDataset shapes: ({input_ids: (512,), token_type_ids: (512,), attention_mask: (512,)}, ()), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>

In [30]:
num_labels = max(df_train["class"]) + 1
kbert_model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", 
                                                        num_labels=num_labels, 
                                                        from_pt=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
kbert_model.compile(optimizer=optimizer, 
              loss=kbert_model.compute_loss, 
              metrics=['accuracy'])
kbert_model.summary()

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 110,621,189
Trainable params: 110,621,189
Non-trainable params: 0
_________________________________________________________________


In [31]:
from tensorflow.keras.callbacks import EarlyStopping

callback_earlystop = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001, # the threshold that triggers the termination (acc should at least improve 0.001)
    patience=2)

kbert_model.fit(
    kbert_train_dataset.shuffle(1000).batch(16), epochs=1, batch_size=16,
    validation_data=kbert_val_dataset.shuffle(1000).batch(16),
    callbacks = [callback_earlystop]
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:377: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


183/273 [===================>..........] - ETA: 1:08:18 - loss: 0.4814 - accuracy: 0.8415

KeyboardInterrupt: ignored